In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# Short Clips - Lateral & Dorsal

This notebook is used to process the manually selected clips of when the fish is swimming between the two jars of stimulus shoals - an inherently risky behaviour. It requires the videos to be turned into frames by ffmpeg code. 

Lateral Clips
- Load in Frames (jpg), give xy prompt, propagate segmentation
- Save basic masks as csv files (0 and 1)
- Create artificially pattered masks as csv files (i.e. add in vertical stripes)
- Treatments: Combine frames and masks to write the various treatments into videos and save the output frames too (as PNG?). 

I will then run these clips through EMD and some of those resulting frames will be run through QCPA to quantify the amount of variation.

Dorsal Clips
- Load in Frames (jpg), give xy prompt, propagate segmentation
- Save basic masks as csv files (0 and 1)
- Save centroid coordinate into a track csv.
I will then load these into Rstudio to analyse the speed and acceleration using trajr. 

<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/sam2/blob/main/notebooks/video_predictor_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 1. SET UP NOTEBOOK

### 1a. Environment Set-Up
* Declare whether using google colab or running locally on jupyter
* Load in packages: os, numpy, torch, matplotlib.pyplot & PIL
* Select device for computation

If running locally using jupyter, first install `sam2` in your environment using the [installation instructions](https://github.com/facebookresearch/sam2#installation) in the repository.

In [1]:
using_colab = False

## PACKAGES ##
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1" # if using Apple MPS, fall back to CPU for unsupported ops
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import pandas as pd
import gc
#for finding the centroid
from shapely.geometry import Polygon
from shapely.ops import polylabel
from shapely.plotting import plot_points, plot_polygon


## SELECT DEVICE FOR COMPUTATION ##
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: cuda


### 1b. Loading the SAM 2 video predictor
* Import checkpoints, models and predictor
* Define plotting helper functions from sam2
* Define other custrom helper functions - registering 'clicks' and appending dictionaries

In [2]:
## LOAD IN SAM2 ##
from sam2.build_sam import build_sam2_video_predictor
sam2_checkpoint = "../checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)


## SAM2 FUNCTIONS ##
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=200): #changing this seems to change the interpretation of the point size as well as the icon size in the output
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='x', s=marker_size, edgecolor='white', linewidth=1.25) # marker shape can be * or x
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='x', s=marker_size, edgecolor='white', linewidth=1.25) 
#n.b. I have removed the code referring to boxes because I don't want to use boxes


# CUSTOM FUNCTIONS ##
def onclick(event): # Define the onclick function to update prompt_pts and clicked flag
    global clicked, prompt_pts_list, prompt_labs_list
    if event.xdata is not None and event.ydata is not None:
        ix, iy = int(event.xdata), int(event.ydata)
        prompt_pts_list.append((ix, iy))
        print(prompt_pts_list)
        
        # Ask user for the label (positive or negative)
        label = input(f"Click registered at: x={ix}, y={iy}. Is this point positive (1) or negative (0)? ")
        while label not in ['0', '1']:
            print("Invalid input. Please enter 1 for positive or 0 for negative.")
            label = input(f"Click registered at: x={ix}, y={iy}. Is this point positive (1) or negative (0)? ")
        
        prompt_labs_list.append(int(label))  # Convert to int32 as needed
        print(prompt_labs_list)
        clicked = True

def updateDict(video_segments, new_dict): #  update dictionaries
    # Find the current max key in D, or start at 0 if D is empty
    max_key = max(map(int, video_segments.keys()), default=-1) + 1
    # Rename the keys in new_dict starting from max_key
    renamed_keys = {str(max_key + i): v for i, v in enumerate(new_dict.values())}
    # Update D
    video_segments.update(renamed_keys)

In [3]:
###### give_prompt redefined to be  allow me to add more clicks

## I ought to change this so I have to specify which obj_id I'm referring to

def give_prompt(clip_dir, frame_name, predictor, inference_state, obj_id=1):

    """
    Displays a specified frame of a clip, waits for a user click, and generates a mask.

    Args:
        clip_dir (str): Path to the folder containing the image.
        frame_name (str): Filename of the frame to display.
        predictor (object): Prediction model for mask generation.
        inference_state (object): Inference state for the predictor.
        obj_id (int, optional): Object ID for mask generation. Default is 1.

    Returns:
        prompt_pts (np.ndarray): Array of selected prompt points.
        prompt_labs (np.ndarray): Array of corresponding point labels.
    """

    global clicked, prompt_pts_list, prompt_labs_list
    prompt_pts_list = []
    prompt_labs_list = []
    done = False
    # Step 1-3. Show figure, clicks, labels

    while (not done):
        fig = plt.figure(figsize=(9,6))
        image_path = os.path.join(clip_dir, clip_frames[0] + '.jpg')  # Adjust index if needed (may need + '.jpg')
        image = mpimg.imread(image_path)
        clicked = False


        imgplot = plt.imshow(image)
        if prompt_pts_list: #runs if point in list
            show_points(prompt_pts, prompt_labs, plt.gca())
            show_mask((out_mask_logits[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])
        plt.show()
        plt.pause(1.0)

        print("Waiting for a response...")
        undo_mask = False
        if prompt_pts_list:
            asked = False
            while not asked:
                plt.pause(0.1)
                add_prompt = input("Do you want to remove mask? (y/n): ").strip().lower()
                if(add_prompt not in ('y', 'n')): print('wrong input') #add stuf here if like
                elif(add_prompt == 'n'):
                    asked = True
                elif(add_prompt == 'y'):
                    prompt_pts_list = []
                    prompt_labs_list = []
                    plt.close()
                    asked = True
                    undo_mask = True
        if undo_mask: continue

        print("Waiting for a response...")
        asked = False
        while not asked:
            plt.pause(0.1)
            add_prompt = input("Do you want to add a click? (y/n): ").strip().lower()
            if(add_prompt not in ('y', 'n')): print('wrong input') #add stuf here if like
            elif(add_prompt == 'n'):
                asked = True
                done = True #probably don't need both
                break
            elif(add_prompt == 'y'):
                asked = True
        if(done): break
        
        clicked = False
        cid = fig.canvas.mpl_connect('button_press_event', onclick)
        print("Waiting for a click...")
        while not clicked:
            plt.pause(0.1)
        plt.close()
        # Interactive wait for the click
        # Small pause to prevent blocking
        
        print("Click registered. Generating mask...")

        prompt_pts = np.array(prompt_pts_list,  dtype=np.int32)
        prompt_labs = np.array(prompt_labs_list,  dtype=np.int32)
        # Step 4. Use prompt to make mask
        _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=0,
        obj_id=obj_id,
        points=prompt_pts, 
        labels=prompt_labs,
        )

    print("Mask generation complete.")
    return prompt_pts, prompt_labs


In [ ]:
#### Testing it out

clip_dir = "/mnt/d/Flora project/Mask_Test/Frames/Botia_lohachata_1_C_1"
prompt_idx = 0
clip_frames = [os.path.join(clip_dir, f) for f in sorted(os.listdir(clip_dir)) if os.path.isfile(os.path.join(clip_dir, f))]
inference_state = predictor.init_state(video_path=clip_dir)


In [ ]:
# /testing it out 
%matplotlib tk

### Step 2: Give Prompt on First Frame
prompt_pts, prompt_labs = give_prompt(
    clip_dir=clip_dir,
    frame_name=clip_frames[0],
    predictor=predictor,
    inference_state=inference_state,
    obj_id=1
)
print(f"Clip 1 / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")


### 1c. Trial set up
* Set video directory
* Set thresholds: clip length, skipped frame tolerance, mask area, and plotting frequency
* Make empty objects called video_segments, clip_directories, and clip_frames
* Standardise and order frame names (i.e. could remove 'frame_' and .jpg extension)

n.b. could also set first prompt here if worked out separately - idx, pts, labs (i.e. frame number, points and labels). This would be useful if, later down the line, I want the for loop to run, not only through clips of a single trial but also, through trials of the same species in the same treatment and beyond. At that point, I may want to import the prompts as a matrix or csv file. 

n.b. would also be better if higher resolution PNG or TIF files were accepted and not just jpg but I might have to change the source code to allow that. 

In [4]:
## VIDEO DIRECTORY ##
# # dorsal
# srcdir = "/mnt/d/Flora project/Clips/Dorsal_clips/Frames" # source directory
# dstdir = "/mnt/d/Flora project/Clips/Dorsal_clips/Output" # destination directory

# lateral
# srcdir = "/mnt/d/Flora project/Mask_Test/Frames" # source directory
# dstdir = "/mnt/d/Flora project/Mask_Test/Output" # destination directory

# Trial
srcdir = "/mnt/d/Flora project/Clips/Dorsal_clips/Frames" # source directory
dstdir = "/mnt/d/Flora project/Clips/Dorsal_clips/Output" # destination directory



clips = [name for name in os.listdir(srcdir) if os.path.isdir(os.path.join(srcdir, name))]
print(clips)


clip_directories = [
    os.path.join(srcdir, name)
    for name in clips
]
print(clip_directories)

out_directories = []
for name in clips:
    out_dir = os.path.join(dstdir, name)
    
    # Check if the output directory exists, and create it if not
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    out_directories.append(out_dir)

print(out_directories)



frame_names = [
    [os.path.splitext(file)[0].replace('frame_', '') for file in os.listdir(folder) if os.path.isfile(os.path.join(folder, file))]
    for folder in clip_directories
]

# ^frame names is a nested list, sort each list/clip frame names separately. 
for frame_list in frame_names:
    frame_list.sort(key=lambda p: int(p))  # Sort each inner list numerically

# frame_names.sort(key=lambda p: int(p))  # Sorting the names by numeric value after removing 'frame_'

print(frame_names)

mask_shape = (1, 1520, 2704) # define mask (frame) dimensions so we can fill with false later
fallback_mask = np.zeros(mask_shape, dtype=bool)


['Botia_lohachata_10_C_R_1_D', 'Botia_lohachata_10_C_R_2_D', 'Botia_lohachata_10_C_R_3_D', 'Botia_lohachata_10_P_L_3_D', 'Botia_lohachata_10_P_L_6_D', 'Botia_lohachata_10_P_R_1_D', 'Botia_lohachata_11_C_L_1_D', 'Botia_lohachata_11_C_L_3_D', 'Botia_lohachata_11_C_L_4_D', 'Botia_lohachata_11_C_L_5_D', 'Botia_lohachata_11_C_R_2_D', 'Botia_lohachata_11_P_L_1_D', 'Botia_lohachata_11_P_R_2_D', 'Botia_lohachata_11_P_R_3_D', 'Botia_lohachata_11_P_R_4_D', 'Botia_lohachata_11_P_R_5_D', 'Botia_lohachata_12_C_23_D', 'Botia_lohachata_12_C_L_10_D', 'Botia_lohachata_12_C_L_11_D', 'Botia_lohachata_12_C_L_12_D', 'Botia_lohachata_12_C_L_17_D', 'Botia_lohachata_12_C_L_19_D', 'Botia_lohachata_12_C_L_1_D', 'Botia_lohachata_12_C_L_3_D', 'Botia_lohachata_12_C_L_4_D', 'Botia_lohachata_12_C_L_5_D', 'Botia_lohachata_12_C_L_7_D', 'Botia_lohachata_12_C_R_14_D', 'Botia_lohachata_12_C_R_16_D', 'Botia_lohachata_12_C_R_18_D', 'Botia_lohachata_12_C_R_20_D', 'Botia_lohachata_12_C_R_22_D', 'Botia_lohachata_12_C_R_24_D',

# 3. PROCESS CLIPS IN FOR LOOP
* 1. Initiate inference_state for each clip referencing its clip_dir
* 2. Add a prompt: for clip1, that will be the prompt_idx, prompt_pts & prompt_labs we estabished in Section 2 but for all successive clips it will be taken from the mask for the last frame of the previous clip. 
* 3. Propagate mask through clip
    * Save to clip_segments{} 
* 4. Save things
    * Append clip_segents to video_segments
    * Save some point in the last mask to use as prompt for next clip - override prompt_pts and prompt_labs 
            note: negative points could be from a mapped out locations of the rocks 
* 5. Reset and delete
    * Reset inference_state
    * Delete subdirectory (or could leave this until the end - doesn't have to be in the for loop)
* 6. Report Progress
    * Render segmentation for the last frame of the clip so I can check where (if ever) the mask lost the fish and to check progress  

## Big Loop for dorsal or lateral clips that allows you to do the prompts first 

I need the code to load in each clip, initiate the inference_state, do the give_prompt function and then save the prompt_pts and prompt_labs along with clip_name so I can refer back to them later. 
Then, I need it to loop through the clips again using those store prompt_pts and prompt_labs for the appropriate clip_name. In this section it should do the rest of the code like propagate through the clip saving the tracks. 

Loop modified so that it processes clips in two passes:
First pass: Iterate through all clips, collect prompt_pts and prompt_labs, and save them with clip_name for future reference.
Second pass: Reload prompt_pts and prompt_labs for each clip and use them for propagation and tracking.
Updated Plan:
Store the prompts in a dictionary {clip_name: (prompt_pts, prompt_labs)}.
Save this dictionary as a JSON or .npz file after the first pass.
Reload the stored prompts in the second pass.

In [5]:
import os
import gc
import json
import numpy as np
import pandas as pd

%matplotlib tk

# Path to store prompt points and labels
prompt_save_path = os.path.join(dstdir, "prompt_data.json")

# Initialize an empty list to store tracking data across all clips
track_coords = []
prompt_data = {}

### **PASS 1: Collect Prompt Points & Labels**
for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    first_frame = clip_frames[0]  
    clip_num = i + 1

    print(f"Clip {clip_num} / Check 1: New Clip Started")

    ### Step 1: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)
    print(f"Clip {clip_num} / Check 2: Inference State Initiated")

    ### Step 2: Give Prompt on First Frame
    prompt_pts, prompt_labs = give_prompt(
        clip_dir=clip_dir,
        frame_name=first_frame,
        predictor=predictor,
        inference_state=inference_state,
        obj_id=1
    )
    
    # Convert numpy arrays to lists for JSON serialization
    prompt_data[clip_name] = {
        "prompt_pts": prompt_pts.tolist(),
        "prompt_labs": prompt_labs.tolist()
    }

    print(f"Clip {clip_num} / Check 3: Prompt saved - points: {prompt_pts.shape}, labels: {prompt_labs.shape}")

    ### Step 3: Reset and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 
    plt.close("all")

# Save prompt data
with open(prompt_save_path, "w") as f:
    json.dump(prompt_data, f)

print("Prompt data saved for all clips.")


### **PASS 2: Process Clips Using Stored Prompts**
for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    output_dir = out_directories[i]
    clip_num = i + 1

    print(f"Clip {clip_num} / Check 4: Processing with stored prompts.")

    ### Step 1: Reload Prompt Data
    prompt_pts = np.array(prompt_data[clip_name]["prompt_pts"], dtype=np.float32)
    prompt_labs = np.array(prompt_data[clip_name]["prompt_labs"], np.int32)
    obj_id = 1

    ### Step 2: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)

    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=0,
    obj_id=obj_id,
    points=prompt_pts,
    labels=prompt_labs,
)

    ### Step 3: Propagate Mask Through Clip
    clip_track = []  

    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(
        inference_state
    ):
        frame_num = clip_frames[out_frame_idx]
        mask_data = out_mask_logits[0] > 0.0 if out_obj_ids else None
        
        if mask_data is not None and np.any(mask_data.cpu().numpy()):
            mask_array = mask_data.cpu().numpy()

            # Ensure mask_array is 2D
            if mask_array.ndim > 2:
                mask_array = np.squeeze(mask_array)

            if mask_array.ndim != 2:
                raise ValueError(f"Unexpected mask shape: {mask_array.shape}")

            y_indices, x_indices = np.where(mask_array)
            center_x, center_y = round(np.mean(x_indices)), round(np.mean(y_indices))

            mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
            mask_path = os.path.join(output_dir, mask_filename)
            np.savetxt(mask_path, mask_array, delimiter=",", fmt="%d")
        else:
            center_x, center_y = None, None

        clip_track.append((frame_num, center_x, center_y))

    # Convert clip-specific tracking data to DataFrame & save it as csv
    clip_track_df = pd.DataFrame(clip_track, columns=['frame', 'x', 'y'])
    clip_csv_path = os.path.join(output_dir, f"{clip_name}_track.csv")
    clip_track_df.to_csv(clip_csv_path, index=False)

    # Append to global tracking dataset
    for row in clip_track:
        track_coords.append((clip_name, *row))

    print(f"Clip {clip_num} data saved to: {clip_csv_path}")

    ### Step 4: Reset and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 

print("All clips processed.")

### FINAL STEP: Save the combined tracking data
track_coords_df = pd.DataFrame(track_coords, columns=['clip', 'frame', 'x', 'y'])
combined_csv_path = os.path.join(dstdir, "Tracks.csv")
track_coords_df.to_csv(combined_csv_path, index=False)

print("Combined tracking data saved to:", combined_csv_path)


Clip 1 / Check 1: New Clip Started


frame loading (JPEG): 100%|██████████| 214/214 [00:10<00:00, 19.69it/s]


Clip 1 / Check 2: Inference State Initiated
Waiting for a response...
Waiting for a response...
Waiting for a click...
[(1210, 623)]
[1]
Click registered. Generating mask...


/tmp/ipykernel_532201/2740669543.py:23: UserWarning: You passed a edgecolor/edgecolors ('white') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='x', s=marker_size, edgecolor='white', linewidth=1.25) # marker shape can be * or x
/tmp/ipykernel_532201/2740669543.py:24: UserWarning: You passed a edgecolor/edgecolors ('white') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='x', s=marker_size, edgecolor='white', linewidth=1.25)


Waiting for a response...
Waiting for a response...
Mask generation complete.
Clip 1 / Check 3: Prompt saved - points: (1, 2), labels: (1,)
Clip 2 / Check 1: New Clip Started


...

Clip 164 data saved to: /mnt/d/Flora project/Clips/Dorsal_clips/Output/Yasuhikotakia_modesta_8_P_R_7_D/Yasuhikotakia_modesta_8_P_R_7_D_track.csv
Clip 165 / Check 4: Processing with stored prompts.


propagate in video: 100%|██████████| 140/140 [01:15<00:00,  1.85it/s]


Clip 165 data saved to: /mnt/d/Flora project/Clips/Dorsal_clips/Output/Yasuhikotakia_modesta_8_P_R_9_D/Yasuhikotakia_modesta_8_P_R_9_D_track.csv
Clip 166 / Check 4: Processing with stored prompts.


propagate in video: 100%|██████████| 81/81 [00:43<00:00,  1.84it/s]

Clip 166 data saved to: /mnt/d/Flora project/Clips/Dorsal_clips/Output/Yasuhikotakia_modesta_9_P_L_4_D/Yasuhikotakia_modesta_9_P_L_4_D_track.csv
All clips processed.
Combined tracking data saved to: /mnt/d/Flora project/Clips/Dorsal_clips/Output/Tracks.csv


## Old/Basic loops & Extra code

separating dorsal and lateral
different tasks

### Loop for dorsal clips

In [ ]:
%matplotlib tk

# Initialize an empty list to store tracking data across all clips
track_coords = []

for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    frame_name = clip_frames[0]  
    clip_num = i + 1
    output_dir = out_directories[i]

    print(f"Clip {clip_num} / Check 1: New Clip Started")

    ### Step 1: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)
    print(f"Clip {clip_num} / Check 2: Inference State Initiated")

    ### Step 2: Give Prompt on First Frame
    prompt_pts, prompt_labs = give_prompt(
        clip_dir=clip_dir,
        frame_name=frame_name,
        predictor=predictor,
        inference_state=inference_state,
        obj_id=1
    )
    print(f"Clip {clip_num} / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")

    ### Step 3: Propagate mask through the clip
    clip_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        clip_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    print(f"Clip {clip_num} / Check 4: Mask Propagation Complete")

    ### Step 4: Store center coordinates
    for frame_num, frame_data in clip_segments.items():
        if frame_data:
            mask = frame_data[1][0]  # Assuming object ID 1
            if mask is not None and np.any(mask):
                # Find the center of the mask
                y_indices, x_indices = np.where(mask)
                center_x, center_y = round(np.mean(x_indices)), round(np.mean(y_indices))

                # Append coordinates
                track_coords.append((clip_name, clip_frames[frame_num], center_x, center_y))

                # # Save mask as CSV
                # mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
                # mask_path = os.path.join(output_dir, mask_filename)
                # np.savetxt(mask_path, mask, delimiter=",", fmt="%d")

            else:
                track_coords.append((clip_name, clip_frames[frame_num], None, None))
                # clip_segments[out_frame_idx][out_obj_id] = np.full(mask_shape, False, dtype=bool)
        else:
            track_coords.append((clip_name, clip_frames[frame_num], None, None))
            # clip_segments[out_frame_idx][out_obj_id] = np.full(mask_shape, False, dtype=bool)

    
    print(f"Clip {clip_num} / Check 5: Masks and Center Coordinates Saved")

    # Reset predictor and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 

    print(f"Clip {clip_num} / Check 6: Reset Complete. Clip {clip_num} finished")

### FINAL STEP: Save the combined data after all clips
# Convert the combined tracking data to a DataFrame
track_coords_df = pd.DataFrame(track_coords, columns=['clip', 'frame', 'x', 'y'])

# Save the combined DataFrame as CSV
combined_csv_path = os.path.join(dstdir, "Tracks.csv")
track_coords_df.to_csv(combined_csv_path, index=False)

print("All clips processed. Combined center coordinates saved to:", combined_csv_path)


In [ ]:
### Dorsal loop with saving the clip track, combined csv of tracks, and masks


%matplotlib tk

# Initialize an empty list to store tracking data across all clips
track_coords = []

for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    frame_name = clip_frames[0]  
    clip_num = i + 1
    output_dir = out_directories[i]

    print(f"Clip {clip_num} / Check 1: New Clip Started")

    ### Step 1: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)
    print(f"Clip {clip_num} / Check 2: Inference State Initiated")

    ### Step 2: Give Prompt on First Frame
    prompt_pts, prompt_labs = give_prompt(
        clip_dir=clip_dir,
        frame_name=frame_name,
        predictor=predictor,
        inference_state=inference_state,
        obj_id=1
    )
    print(f"Clip {clip_num} / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")

    ### Step 3: Propagate mask through the clip Saving tracks and masks

    ## 3a - tracks
    # this should check the dimensions of the mask (unlike above - idk why it worked before)
    clip_track = []  

    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        frame_num = clip_frames[out_frame_idx]
        mask_data = out_mask_logits[0] > 0.0 if out_obj_ids else None
        
        if mask_data is not None and np.any(mask_data.cpu().numpy()):
            mask_array = mask_data.cpu().numpy()

            # Ensure mask_array is 2D
            if mask_array.ndim > 2:
                mask_array = np.squeeze(mask_array)

            if mask_array.ndim != 2:
                raise ValueError(f"Unexpected mask shape: {mask_array.shape}")

            y_indices, x_indices = np.where(mask_array)
            center_x, center_y = round(np.mean(x_indices)), round(np.mean(y_indices))

            mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
            mask_path = os.path.join(output_dir, mask_filename)
            np.savetxt(mask_path, mask_array, delimiter=",", fmt="%d")
        else:
            center_x, center_y = None, None

        # Append to clip-specific tracking data
        clip_track.append((frame_num, center_x, center_y))

    # Convert clip-specific tracking data to DataFrame & save it as csv
    clip_track_df = pd.DataFrame(clip_track, columns=['frame', 'x', 'y'])
    clip_csv_path = os.path.join(output_dir, f"{clip_name}_track.csv")
    clip_track_df.to_csv(clip_csv_path, index=False)

    # Append to global tracking dataset, adding `clip_name`
    for row in clip_track:
        track_coords.append((clip_name, *row))  # Expands (frame_num, center_x, center_y)

    print(f"Clip {clip_num} data saved to: {clip_csv_path}")
    print(f"Clip {clip_num} / Check 4: Masks and Center Coordinates Saved")

    ### Step 4: Reset and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 

    print(f"Clip {clip_num} / Check 6: Reset Complete. Clip {clip_num} finished")

### FINAL STEP: Save the combined data after all clips
# Convert the combined tracking data to a DataFrame
track_coords_df = pd.DataFrame(track_coords, columns=['clip', 'frame', 'x', 'y'])

# Save the combined DataFrame as CSV
combined_csv_path = os.path.join(dstdir, "Tracks.csv")
track_coords_df.to_csv(combined_csv_path, index=False)

print("All clips processed. Combined center coordinates saved to:", combined_csv_path)

### Loop for lateral clips - masks

In [ ]:
%matplotlib tk

# Initialize an empty list to store tracking data across all clips

for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    frame_name = clip_frames[0]  
    clip_num = i + 1
    output_dir = out_directories[i]

    print(f"Clip {clip_num} / Check 1: New Clip Started")

    ### Step 1: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)
    print(f"Clip {clip_num} / Check 2: Inference State Initiated")

    ### Step 2: Give Prompt on First Frame
    prompt_pts, prompt_labs = give_prompt(
        clip_dir=clip_dir,
        frame_name=frame_name,
        predictor=predictor,
        inference_state=inference_state,
        obj_id=1
    )
    print(f"Clip {clip_num} / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")

    ### Step 3: Propagate mask through the clip
    clip_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        clip_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    print(f"Clip {clip_num} / Check 4: Mask Propagation Complete")

    ### Step 4: Store masks as csv
    for frame_num, frame_data in clip_segments.items():
        if frame_data:
            mask = frame_data[1][0]  # Assuming object ID 1
            if mask is not None and np.any(mask):
                
                # Save mask as CSV
                mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
                mask_path = os.path.join(output_dir, mask_filename)
                np.savetxt(mask_path, mask, delimiter=",", fmt="%d")

            else:
                clip_segments[out_frame_idx][out_obj_id] = np.full(mask_shape, False, dtype=bool)
        else:
            clip_segments[out_frame_idx][out_obj_id] = np.full(mask_shape, False, dtype=bool)

    print(f"Clip {clip_num} / Check 5: Masks and Center Coordinates Saved")

    # Reset predictor and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 

    print(f"Clip {clip_num} / Check 6: Reset Complete. Clip {clip_num} finished")


print("All clips processed.")


### Adding mask altering - mean luminance 

In [ ]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import re

%matplotlib tk

# Function to extract frame number from filename
def extract_frame_number(filename):
    match = re.search(r'frame_(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

# Initialize processing for all clips
for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    frame_name = clip_frames[0]  
    clip_num = i + 1
    output_dir = out_directories[i]

    print(f"Clip {clip_num} / Check 1: New Clip Started")

    ### Step 1: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)
    print(f"Clip {clip_num} / Check 2: Inference State Initiated")

    ### Step 2: Give Prompt on First Frame
    prompt_pts, prompt_labs = give_prompt(
        clip_dir=clip_dir,
        frame_name=frame_name,
        predictor=predictor,
        inference_state=inference_state,
        obj_id=1
    )
    print(f"Clip {clip_num} / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")

    ### Step 3: Propagate mask through the clip
    clip_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        clip_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    print(f"Clip {clip_num} / Check 4: Mask Propagation Complete")

    ### Step 4: Store masks as csv
    for frame_num, frame_data in clip_segments.items():
        if frame_data:
            mask = frame_data[1][0]  # Assuming object ID 1
            if mask is not None and np.any(mask):
                
                # Save mask as CSV
                mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
                mask_path = os.path.join(output_dir, mask_filename)
                np.savetxt(mask_path, mask, delimiter=",", fmt="%d")

    print(f"Clip {clip_num} / Check 5: Masks CSVs Saved")



    print(f"Clip {clip_num} / Check 6: Generating Videos...")

    ### Step 5: Generate Videos using Frames and Masks
    frame_dir = clip_dir  # Use original frames
    mask_dir = output_dir  # Where masks were saved
    dst_dir = output_dir  # Save videos in same directory

    # Sort frames and masks
    frames = sorted(os.listdir(frame_dir))
    masks = sorted(os.listdir(mask_dir), key=extract_frame_number)

    # Read first frame to get dimensions
    sample_frame = cv2.imread(os.path.join(frame_dir, frames[0]))
    height, width, _ = sample_frame.shape
    fps = 60  # Adjust if needed

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video1 = cv2.VideoWriter(os.path.join(dst_dir, 'unaltered_mask.mp4'), fourcc, fps, (width, height))
    video2 = cv2.VideoWriter(os.path.join(dst_dir, 'mean_luminance_fish.mp4'), fourcc, fps, (width, height))
    video3 = cv2.VideoWriter(os.path.join(dst_dir, 'mean_luminance_background.mp4'), fourcc, fps, (width, height))
    video4 = cv2.VideoWriter(os.path.join(dst_dir, 'mean_luminance_both.mp4'), fourcc, fps, (width, height))

    for frame_name, mask_name in zip(frames, masks):
        frame_path = os.path.join(frame_dir, frame_name)
        mask_path = os.path.join(mask_dir, mask_name)

        # Read frame and mask
        frame = cv2.imread(frame_path)
        mask = pd.read_csv(mask_path, header=None).values  # Convert CSV to NumPy array

        # Compute mean luminance
        fish_pixels = frame[mask == 1]
        background_pixels = frame[mask == 0]

        mean_fish = (
            np.mean(fish_pixels, axis=0).astype(np.uint8)
            if len(fish_pixels) > 0
            else np.array([0, 0, 0], dtype=np.uint8)
        )
        mean_background = (
            np.mean(background_pixels, axis=0).astype(np.uint8)
            if len(background_pixels) > 0
            else np.array([0, 0, 0], dtype=np.uint8)
        )

        # Create mean luminance images
        mean_fish_img = np.full(frame.shape, mean_fish, dtype=np.uint8)
        mean_background_img = np.full(frame.shape, mean_background, dtype=np.uint8)

        # Apply treatments
        unaltered_mask = frame.copy()
        mean_fish_treatment = frame.copy()
        mean_background_treatment = frame.copy()
        mean_both_treatment = frame.copy()

        # Apply modifications
        mean_fish_treatment[mask == 1] = mean_fish  # Replace fish with mean luminance
        mean_background_treatment[mask == 0] = mean_background  # Replace background with mean luminance
        mean_both_treatment[mask == 1] = mean_fish  # Replace fish
        mean_both_treatment[mask == 0] = mean_background  # Replace background

        # Write frames to videos
        video1.write(unaltered_mask)
        video2.write(mean_fish_treatment)
        video3.write(mean_background_treatment)
        video4.write(mean_both_treatment)

    # Release videos
    video1.release()
    video2.release()
    video3.release()
    video4.release()

    print(f"Clip {clip_num} / Check 7: Videos Saved")

    # Reset predictor and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 
    print(f"Clip {clip_num} / Check 7: Reset complete. Clip {clip_num} finished.")

print("All clips processed.")


#### Extra unrelated code

In [ ]:
# render the segmentation results every few frames
vis_frame_stride = 50
plt.close("all")
for out_frame_idx in range(0, len(clip_frames), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(Image.open(os.path.join(clip_dir, clip_frames[out_frame_idx]) + ".jpg"))
    for out_obj_id, out_mask in clip_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

track_coords_df

###########################
## shapely center

# Step 2: Create a list of coordinates for the polygon
mask_coords = [(x_indices[i], y_indices[i]) for i in range(len(y_indices))]

# Step 3: Create a Shapely Polygon and compute the centroid
polygon = Polygon(mask_coords)
polylabel_point = polylabel(polygon, tolerance=1)


######################

coordinates = []
for i in range(len(y_indices)):
    current = (x_indices[i], y_indices[i])
    coordinates.append(current)
coordinates = tuple(coordinates)
#
polygon = Polygon(coordinates)
rep_point = polygon.representative_point() # 2. representative point method
rep_x, rep_y = rep_point.x, rep_point.y

print(f"rep_x: {rep_x}, rep_y: {rep_y}")

print(rep_point)

# In action - for Katie and Anna

### Initial Trial Set-up

Katie (18/02/2025) - Dorsal masks from long-ish clip to use to help make a skeleton for 3D tracking
Anna (04/03/2025) - Dorsal tracks of multiple individuals to test occlusion. I didn't do them simultaneously, I just did them one at a time so it wasn't a true test

In [ ]:
### for katie
# srcdir = "/mnt/c/Users/sjoh5476/Desktop/Katie/Frames" 
# dstdir = "/mnt/c/Users/sjoh5476/Desktop/Katie/Output" 

### for anna
srcdir = "/mnt/c/Users/sjoh5476/Desktop/Anna/AnnaClipFrames"
dstdir = "/mnt/c/Users/sjoh5476/Desktop/Anna/AnnaClipFrames/Output"


import os

# Scan all the JPEG frame names in this directory and remove 'frame_' and the extension
frame_names = [
    os.path.splitext(p)[0].replace('frame_', '')  # Remove 'frame_' and extension
    for p in os.listdir(srcdir)
    if os.path.splitext(p)[-1].lower() in [".jpg", ".jpeg"]
]

# Sort the names by numeric value (assuming the names are numeric after removing 'frame_')
frame_names.sort(key=lambda p: int(p))

print(frame_names)


mask_shape = (1, 1520, 2704) # define mask (frame) dimensions so we can fill with false later
fallback_mask = np.zeros(mask_shape, dtype=bool)

clip_dirs = [
    os.path.join(srcdir, d) for d in os.listdir(srcdir)
    if os.path.isdir(os.path.join(srcdir, d)) and d.startswith("clip_")
]

print(clip_dirs)

In [ ]:
import os

def split_clip_into_subclips(video_dir, frame_names, clip_len=500):
    clip_num = 1  # Start numbering the clips
    prompt_idx = 0  # Starting index for the frames
    clip_dirs = {}  # Dictionary to store the clip directories

    while prompt_idx < len(frame_names):
        # Step 1. Create a new subdirectory for the current clip 
        clip_fr = frame_names[prompt_idx:prompt_idx + clip_len]
        if not clip_fr:  # Stop if no frames are left!!!!
            print("All frames processed.")
            break
        
        clip_dir = os.path.join(video_dir, f"clip_{clip_num}")
        
        # Check if the directory already exists, if not, create it
        if not os.path.exists(clip_dir):
            os.mkdir(clip_dir)
        else:
            print(f"Directory already exists: {clip_dir}")
        
        # Store the clip_dir in the dictionary
        clip_dirs[clip_num] = clip_dir
        
        # Step 2. Populate the subdirectory with hard links to the relevant frames
        for frame in clip_fr:
            src_path = os.path.join(video_dir, f"{frame}.jpg")
            dst_path = os.path.join(clip_dir, os.path.basename(src_path))
            
            # Check if the source file exists before creating a hard link
            if os.path.exists(src_path):
                os.link(src_path, dst_path)  # Create a hard link
            else:
                print(f"Warning: Source file does not exist: {src_path}")
        
        # Checkpoint1
        print(f"Clip {clip_num} / Check1: Clip Creation Complete")
        
        # Step 3. Move to the next batch of frames
        prompt_idx += clip_len
        clip_num += 1  # Increment the clip number for the next subclip

    # Print the clip directories for verification
    print("Clip directories created:", clip_dirs)
    return clip_dirs  # Return the dictionary containing all clip directories

# Example usage
# srcdir = "/mnt/c/Users/sjoh5476/Desktop/Katie/Frames"
srcdir = "/mnt/c/Users/sjoh5476/Desktop/Anna/AnnaClipFrames"
clip_dirs = split_clip_into_subclips(srcdir, frame_names)


In [ ]:
%matplotlib tk

import os
import gc
import numpy as np
import pandas as pd

# Define a name for the video (could be the name of the video file or anything you like)
video_name = "Zebrafish_clip_blender"  # Replace with your video name

# Initialize an empty list to store tracking data across all frames
track_coords = []

# Loop through all clip folders from clip1 to clip10
for clip_num in range(1, 21):  # Loop through clip1 to clip10
    clip_dir = clip_dirs[clip_num]
    
    # Check if the folder exists
    if not os.path.exists(clip_dir):
        print(f"Folder {clip_dir} does not exist. Skipping...")
        continue

    inference_state = predictor.init_state(video_path=clip_dir)

    # Get the frame names in the current clip folder
    # Get the sorted list of frame names, excluding the .jpg extension
    frame_names = sorted([
        os.path.splitext(p)[0]  # Remove the .jpg extension
        for p in os.listdir(clip_dir)
        if os.path.splitext(p)[-1].lower() in [".jpg", ".jpeg"]
    ])

    print(f"Processing {clip_dir} with {len(frame_names)} frames.")

    # Iterate through all frames in the directory
    for i, frame_name in enumerate(frame_names):
        print(f"Processing frame {i+1} / {len(frame_names)}: {frame_name}")

        ### Step 2: Give Prompt on Current Frame
        prompt_pts, prompt_labs = give_prompt(
            clip_dir=clip_dir,
            frame_name=frame_name,
            predictor=predictor,
            inference_state=inference_state,
            obj_id=1  # Assuming you're tracking object ID 1
        )
        print(f"Prompt given for {frame_name}: points: {prompt_pts}, labels: {prompt_labs}")

        ### Step 3: Propagate mask for the current frame
        clip_segments = {}
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
            clip_segments[out_frame_idx] = {
                out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }

        print(f"Mask propagation complete for {frame_name}")

        ### Step 4: Store center coordinates for the current frame
        if clip_segments:
            mask = clip_segments.get(1, {}).get(0, None)  # Assuming object ID 1
            if mask is not None and np.any(mask):
                # Find the center of the mask
                y_indices, x_indices = np.where(mask)
                center_x, center_y = round(np.mean(x_indices)), round(np.mean(y_indices))

                # Append coordinates
                track_coords.append((video_name, frame_name, center_x, center_y))
            else:
                track_coords.append((video_name, frame_name, None, None))

        else:
            track_coords.append((video_name, frame_name, None, None))

        print(f"Center coordinates saved for {frame_name}")

        ### Step 5: Store masks as CSV for the current frame
        if clip_segments:
            mask = clip_segments.get(1, {}).get(0, None)  # Assuming object ID 1
            if mask is not None and np.any(mask):
                mask_filename = f"frame_{i+1}_mask.csv"
                mask_path = os.path.join(dstdir, mask_filename)
                np.savetxt(mask_path, mask, delimiter=",", fmt="%d")

        print(f"Mask saved for {frame_name}")

        # Reset predictor and clear memory
        gc.collect()
        predictor.reset_state(inference_state)  # Reset the state for the next frame

# FINAL STEP: Save the combined data after all frames
track_coords_df = pd.DataFrame(track_coords, columns=['clip', 'frame', 'x', 'y'])
combined_csv_path = os.path.join(dstdir, "Track.csv")
track_coords_df.to_csv(combined_csv_path, index=False)

print(f"All frames processed. Combined center coordinates saved to: {combined_csv_path}")


In [ ]:
### Looping through clips of the same video!

%matplotlib tk

# Initialize an empty list to store tracking data across all clips
track_coords = []

for i, clip_name in enumerate(clips):
    clip_dir = clip_directories[i]  
    clip_frames = frame_names[i]
    frame_name = clip_frames[0]  
    clip_num = i + 1
    output_dir = out_directories[i]

    print(f"Clip {clip_num} / Check 1: New Clip Started")

    ### Step 1: Initiate Inference State
    inference_state = predictor.init_state(video_path=clip_dir)
    print(f"Clip {clip_num} / Check 2: Inference State Initiated")

    ### Step 2: Give Prompt on First Frame
    prompt_pts, prompt_labs = give_prompt(
        clip_dir=clip_dir,
        frame_name=frame_name,
        predictor=predictor,
        inference_state=inference_state,
        obj_id=1
    )
    print(f"Clip {clip_num} / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")

    ### Step 3: Propagate mask through the clip
    clip_segments = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        clip_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

    print(f"Clip {clip_num} / Check 4: Mask Propagation Complete")

    ### Step 4: Store center coordinates
    for frame_num, frame_data in clip_segments.items():
        if frame_data:
            mask = frame_data[1][0]  # Assuming object ID 1
            if mask is not None and np.any(mask):
                # Find the center of the mask
                y_indices, x_indices = np.where(mask)
                center_x, center_y = round(np.mean(x_indices)), round(np.mean(y_indices))

                # Append coordinates
                track_coords.append((clip_name, clip_frames[frame_num], center_x, center_y))

            else:
                track_coords.append((clip_name, clip_frames[frame_num], None, None))
        else:
            track_coords.append((clip_name, clip_frames[frame_num], None, None))

    print(f"Clip {clip_num} / Check 5: Masks and Center Coordinates Saved")

    ### Step 5: Store masks as csv
    for frame_num, frame_data in clip_segments.items():
        if frame_data:
            mask = frame_data[1][0]  # Assuming object ID 1
            if mask is not None and np.any(mask):
                
                # Save mask as CSV
                mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
                mask_path = os.path.join(output_dir, mask_filename)
                np.savetxt(mask_path, mask, delimiter=",", fmt="%d")

            else:
                clip_segments[out_frame_idx][out_obj_id] = np.full(mask_shape, False, dtype=bool)
        else:
            clip_segments[out_frame_idx][out_obj_id] = np.full(mask_shape, False, dtype=bool)

    print(f"Clip {clip_num} / Check 5: Masks and Center Coordinates Saved")

    # Reset predictor and clear memory
    gc.collect()
    predictor.reset_state(inference_state) 

    print(f"Clip {clip_num} / Check 6: Reset Complete. Clip {clip_num} finished")

### FINAL STEP: Save the combined data after all clips
# Convert the combined tracking data to a DataFrame
track_coords_df = pd.DataFrame(track_coords, columns=['clip', 'frame', 'x', 'y'])

# Save the combined DataFrame as CSV
combined_csv_path = os.path.join(dstdir, "Tracks.csv")
track_coords_df.to_csv(combined_csv_path, index=False)

print("All clips processed. Combined center coordinates saved to:", combined_csv_path)

In [ ]:
### Removing loop

%matplotlib tk

import os
import gc
import numpy as np
import pandas as pd

# Specify the clip directory you want to process
clip_name="Anna"
# clip_dir = f"/mnt/c/Users/sjoh5476/Desktop/Katie/Frames/{clip_name}"
# output_dir = f"/mnt/c/Users/sjoh5476/Desktop/Katie/Output/{clip_name}"

clip_dir = "/mnt/c/Users/sjoh5476/Desktop/Anna/AnnaClipFrames"
output_dir = "/mnt/c/Users/sjoh5476/Desktop/Anna/AnnaClipFrames/Output"



# Get the sorted list of frame names, excluding the .jpg extension
frame_names = sorted([
    os.path.splitext(p)[0]  # Remove the .jpg extension
    for p in os.listdir(clip_dir)
    if os.path.splitext(p)[-1].lower() in [".jpg", ".jpeg"]
])

if not frame_names:
    print(f"No valid frames found in {clip_dir}. Exiting.")
    exit()

frame_name = frame_names[0]  # First frame for prompting
track_coords = []  # Store tracking data

print(f"{clip_name} / Check 1: New Clip Started")

### Step 1: Initiate Inference State
inference_state = predictor.init_state(video_path=clip_dir)
print(f"{clip_name} / Check 2: Inference State Initiated")

### Step 2: Give Prompt on First Frame
prompt_pts, prompt_labs = give_prompt(
    clip_dir=clip_dir,
    frame_name=frame_name,
    predictor=predictor,
    inference_state=inference_state,
    obj_id=1
)
print(f"{clip_name} / Check 3: Prompt given - points: {prompt_pts}, labels: {prompt_labs}")

### Step 3: Propagate mask through the clip
clip_segments = {}
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
    clip_segments[out_frame_idx] = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

print(f"{clip_name} / Check 4: Mask Propagation Complete")

### Step 4: Store center coordinates
for frame_num, frame_data in clip_segments.items():
    if frame_data:
        mask = frame_data[1][0]  # Assuming object ID 1
        if mask is not None and np.any(mask):
            # Find the center of the mask
            y_indices, x_indices = np.where(mask)
            center_x, center_y = round(np.mean(x_indices)), round(np.mean(y_indices))

            # Append coordinates
            track_coords.append((clip_name, frame_names[frame_num], center_x, center_y))
        else:
            track_coords.append((clip_name, frame_names[frame_num], None, None))
    else:
        track_coords.append((clip_name, frame_names[frame_num], None, None))

print(f"{clip_name} / Check 5: Masks and Center Coordinates Saved")

### Step 5: Store masks as CSV
for frame_num, frame_data in clip_segments.items():
    if frame_data:
        mask = frame_data[1][0]  # Assuming object ID 1
        if mask is not None and np.any(mask):
            # Save mask as CSV
            #mask_filename = f"{clip_name}_frame_{frame_num}_mask.csv"
            mask_filename = f"{frame_names[frame_num]}_mask.csv"
            mask_path = os.path.join(output_dir, mask_filename)
            np.savetxt(mask_path, mask, delimiter=",", fmt="%d")

print(f"{clip_name} / Check 6: Masks saved successfully.")

# Reset predictor and clear memory
gc.collect()
predictor.reset_state(inference_state)

print(f"{clip_name} / Check 7: Reset Complete. Clip finished.")

### FINAL STEP: Save the tracking data
track_coords_df = pd.DataFrame(track_coords, columns=['clip', 'frame', 'x', 'y'])
combined_csv_path = os.path.join(output_dir, f"{clip_name}_Track.csv")
track_coords_df.to_csv(combined_csv_path, index=False)

print(f"Tracking data for {clip_name} saved to:", combined_csv_path)


In [ ]:
# Step 5:  Render Segmentation Results
for out_frame_idx in range(0, len(frame_names), 25):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(Image.open(os.path.join(clip_dir, frame_names[out_frame_idx]) + '.jpg'))
    for out_obj_id, out_mask in clip_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

#checkpoint5
print("Check5: Rendering Frames Complete")

In [ ]:
plt.close("all")

In [ ]:
## Save output as a video

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Define output directory for frames
output_frames_dir = "/mnt/c/Users/sjoh5476/Desktop/Anna/Rendered_Frames"
os.makedirs(output_frames_dir, exist_ok=True)

# Step 5: Render and save segmentation results
frame_list = []
for out_frame_idx in range(0, len(frame_names), 1):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_title(f"frame {out_frame_idx}")

    # Load and display frame
    img_path = os.path.join(clip_dir, frame_names[out_frame_idx]) + ".jpg"
    img = Image.open(img_path)
    ax.imshow(img)

    # Overlay segmentation mask
    for out_obj_id, out_mask in clip_segments[out_frame_idx].items():
        show_mask(out_mask, ax, obj_id=out_obj_id)

    # Save frame as an image
    frame_path = os.path.join(output_frames_dir, f"frame_{out_frame_idx:04d}.png")
    fig.savefig(frame_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)  # Close figure to free memory

    frame_list.append(frame_path)

# Checkpoint
print("Check5: Rendering Frames Complete")

# Step 6: Convert saved images to a video
output_video_path = "/mnt/c/Users/sjoh5476/Desktop/Anna/2_segmentation_output.mp4"
frame_rate = 30  # Adjust as needed

# Read first image to get dimensions
first_frame = cv2.imread(frame_list[0])
height, width, layers = first_frame.shape

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

# Add frames to video
for frame_path in frame_list:
    img = cv2.imread(frame_path)
    video_writer.write(img)

video_writer.release()
print(f"Video saved at: {output_video_path}")
